### This example illustrates how to use mlflow to track parameters, metrics, artifacts and models while training a simple MNIST model.

In [3]:
import mlflow.fastai
from fastai.vision.all import (
    CategoryBlock,
    DataBlock,
    GrandparentSplitter,
    ImageBlock,
    PILImage,
    URLs,
)
from fastai.vision.all import cnn_learner, get_image_files, parent_label, resnet18, untar_data
import numpy as np

#### Splitting data between training and validation

In [4]:
splitter = GrandparentSplitter(train_name="training", valid_name="testing")

# Prepare DataBlock which is a generic container to quickly build Datasets and DataLoaders
mnist = DataBlock(
    blocks=(ImageBlock(PILImage), CategoryBlock),
    get_items=get_image_files,
    splitter=splitter,
    get_y=parent_label,
)

#### Downloading MNIST data set and creating DataLoader from Datablock

In [5]:
data_path = untar_data(URLs.MNIST)
image_files = get_image_files(data_path)
print('Data located at:', data_path)

data = mnist.dataloaders(data_path, bs=256, num_workers=0)

data.train.show_batch()


Data located at: /home/azureuser/.fastai/data/mnist_png


##### Creating signature

In [9]:
import numpy as np
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, TensorSpec

input_schema = Schema([
  TensorSpec(np.dtype(np.uint8), (-1, -1, -1, 3)),
])
output_schema = Schema([
  TensorSpec(np.dtype(np.float32), (-1, 2)),
])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

##### Enabling autologging from mlflow

In [ ]:
# mlflow.fastai.autolog()

##### Create Learner model

In [10]:

learn = cnn_learner(data, resnet18)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/azureuser/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

##### Using different loading function

In [11]:
%%writefile fastai_model_loader.py
import os
import numpy as np
import pandas as pd

class _FastaiModelWrapperPatch:
    def __init__(self, learner):
        self.learner = learner

    def predict(self, dataframe):
        dl = self.learner.dls.test_dl(dataframe)
        preds, _ = self.learner.get_preds(dl=dl)
        return pd.Series(map(np.array, preds.numpy())).to_frame('predictions')

def _load_model(path):
    from fastai.learner import load_learner

    return load_learner(os.path.abspath(path))

def _load_pyfunc(path):
    """
    Load PyFunc implementation. Called by ``pyfunc.load_pyfunc``.
    :param path: Local filesystem path to the MLflow Model with the ``fastai`` flavor.
    """
    print("Model loaded from:",path)
    return _FastaiModelWrapperPatch(_load_model(path))

Writing fastai_model_loader.py


##### Training model

In [21]:
saved_model = "/home/azureuser/.fastai/model.fastai"

with mlflow.start_run() as run:
    # Train and fit with default or supplied command line arguments
    learn.fit_one_cycle(1, 0.1)
    learn.export(saved_model)
    model = mlflow.pyfunc.log_model("model", 
                                    registered_model_name="digits_cnn_model",
                                    data_path=saved_model, 
                                    code_path=["./fastai_model_loader.py"], 
                                    loader_module="fastai_model_loader", 
                                    signature=signature)

epoch,train_loss,valid_loss,time
0,nan,None,00:00


/anaconda/envs/fastai-example/lib/python3.7/site-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")
Successfully registered model 'digits_cnn_model'.
2022/02/25 19:19:03 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: digits_cnn_model, version 1
Created version '1' of model 'digits_cnn_model'.


#### Loading trained model from the run

In [40]:
model_uri = "runs:/{}/model".format(run.info.run_id)
loaded_model = mlflow.pyfunc.load_model(model_uri)
model_uri

Model loaded from: /tmp/tmp355uj81s/model/data/model.fastai


'runs:/11df06dd-907f-4134-a537-9ee435c5b47a/model'

 #### Creating a sample batch for predcition to a rest endpoint

In [32]:
from fastai.vision.core import load_image

# sample_img = load_image(image_files[1])
sample_img_arr = np.array(data.train_ds[0][0])

sample_batch = sample_img_arr.reshape(1, sample_img.height, sample_img.width,3)

In [47]:
data.train_ds[0][0]

##### Local prediction

In [48]:
loaded_model.predict(sample_batch)

,predictions
0,"[8.421067e-07, 2.8983236e-06, 1.3368848e-06, 0.9994823, 1.6431642e-06, 8.523096e-05, 7.7165737e-07, 4.1304946e-07, 0.00041827984, 6.3882144e-06]"


### Running the model as a local endpoint

In [45]:
import json
import requests

inference_request = {
    "instances": sample_batch.tolist()
}

In [46]:
endpoint = "http://localhost:5000/invocations"
response = requests.post(endpoint, json=inference_request)

response.json()

[{'predictions': [8.421067150266026e-07,
   2.8983236006752122e-06,
   1.3368847930905758e-06,
   0.999482274055481,
   1.6431641824965482e-06,
   8.523096039425582e-05,
   7.716573691141093e-07,
   4.130494630771864e-07,
   0.0004182798438705504,
   6.388214387698099e-06]}]